In [1]:
import numpy as np
import pandas as pd
from scipy.stats import pearsonr
import math
import re
import os
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import plotly
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
!pwd

/Users/shijunyi/Desktop/学习资料/U of Melbourne/2019 Sem2/Data Science Project/[6] code/python


In [2]:
pd.options.display.precision = 2
pd.options.display.max_columns = 50
matplotlib.rcParams['figure.figsize'] = [15, 10]
init_notebook_mode(connected=True)

In [3]:
src_countries = ['China', 'India', 'Indonesia', 'Malaysia']
final_result_df = pd.DataFrame(
                            columns = ['Source', 'Destination', 'Predictor', 
                                       'Shift', 'Correlation', 'P-value', 'Tested Corralation'])

In [4]:
for src in src_countries:
    #read data
    factors_df = pd.read_csv('data/grouped_'+src+'.csv')
    
    #calculate correalation and p-value for each destination:
    corr_dic={}
    pvalue_dic = {}
    tested_dic = {}
    dest_countries = ['United States of America', 'United Kingdom', 'Australia', 'Canada', 'New Zealand']

    #filter record for each destination:
    for dest in dest_countries:
        dest_factors = factors_df[factors_df['destination_country']== dest]
        dest_factors = dest_factors.drop(['source_country', 'destination_country', 'dst_distance'],axis=1)
        
        #shift predictors by 1 and 2 years:
        dest_factors.sort_values('year',inplace=True)
        cols = dest_factors.columns
        dest_factors = dest_factors.join(dest_factors[cols].shift(1), rsuffix='_1y_shift')
        dest_factors = dest_factors.join(dest_factors[cols].shift(2), rsuffix='_2y_shift')
        dest_factors = dest_factors.join(dest_factors[cols].shift(3), rsuffix='_3y_shift')
        dest_factors = dest_factors.join(dest_factors[cols].shift(4), rsuffix='_4y_shift')
        dest_factors = dest_factors.join(dest_factors[cols].shift(5), rsuffix='_5y_shift')
 
        #rearrange cols:
        new_col = [
        'dst_students_count',
        'year', 'year_1y_shift', 'year_2y_shift', 'year_3y_shift', 'year_4y_shift', 'year_5y_shift',
        'dst_average_wage', 'dst_average_wage_1y_shift', 'dst_average_wage_2y_shift',
        'dst_average_wage_3y_shift', 'dst_average_wage_4y_shift', 'dst_average_wage_5y_shift',
        'dst_rate_of_one_usd', 'dst_rate_of_one_usd_1y_shift', 'dst_rate_of_one_usd_2y_shift',
        'dst_rate_of_one_usd_3y_shift', 'dst_rate_of_one_usd_4y_shift', 'dst_rate_of_one_usd_5y_shift',
        'dst_cpi', 'dst_cpi_1y_shift', 'dst_cpi_2y_shift',
        'dst_cpi_3y_shift', 'dst_cpi_4y_shift', 'dst_cpi_5y_shift',
        'dst_qoe_top20', 'dst_qoe_top20_1y_shift' , 'dst_qoe_top20_2y_shift',
        'dst_qoe_top20_3y_shift', 'dst_qoe_top20_4y_shift', 'dst_qoe_top20_5y_shift',
        'dst_qoe_top100', 'dst_qoe_top100_1y_shift', 'dst_qoe_top100_2y_shift',
        'dst_qoe_top100_3y_shift', 'dst_qoe_top100_4y_shift', 'dst_qoe_top100_5y_shift',
        'dst_qoe_top200', 'dst_qoe_top200_1y_shift', 'dst_qoe_top200_2y_shift',
        'dst_qoe_top200_3y_shift', 'dst_qoe_top200_4y_shift', 'dst_qoe_top200_5y_shift',
        'dst_qoe_top500', 'dst_qoe_top500_1y_shift', 'dst_qoe_top500_2y_shift',
        'dst_qoe_top500_3y_shift', 'dst_qoe_top500_4y_shift', 'dst_qoe_top500_5y_shift',
        'src_population', 'src_population_1y_shift', 'src_population_2y_shift',
        'src_population_3y_shift', 'src_population_4y_shift', 'src_population_5y_shift',
        'src_qoe_top20', 'src_qoe_top20_1y_shift', 'src_qoe_top20_2y_shift',
        'src_qoe_top20_3y_shift', 'src_qoe_top20_4y_shift', 'src_qoe_top20_5y_shift',
        'src_qoe_top100', 'src_qoe_top100_1y_shift', 'src_qoe_top100_2y_shift',
        'src_qoe_top100_3y_shift', 'src_qoe_top100_4y_shift', 'src_qoe_top100_5y_shift',
        'src_qoe_top200', 'src_qoe_top200_1y_shift', 'src_qoe_top200_2y_shift',
        'src_qoe_top200_3y_shift', 'src_qoe_top200_4y_shift', 'src_qoe_top200_5y_shift',
        'src_qoe_top500', 'src_qoe_top500_1y_shift', 'src_qoe_top500_2y_shift',
        'src_qoe_top500_3y_shift', 'src_qoe_top500_4y_shift', 'src_qoe_top500_5y_shift',
        'src_gni', 'src_gni_1y_shift', 'src_gni_2y_shift', 
        'src_gni_3y_shift', 'src_gni_4y_shift', 'src_gni_5y_shift',
        'src_gdp', 'src_gdp_1y_shift', 'src_gdp_2y_shift', 
        'src_gdp_3y_shift', 'src_gdp_4y_shift', 'src_gdp_5y_shift',
        'src_rate_of_one_usd', 'src_rate_of_one_usd_1y_shift', 'src_rate_of_one_usd_2y_shift', 
        'src_rate_of_one_usd_3y_shift', 'src_rate_of_one_usd_4y_shift', 'src_rate_of_one_usd_5y_shift',
        'src_tertiary_enrolment', 'src_tertiary_enrolment_1y_shift', 'src_tertiary_enrolment_2y_shift', 
        'src_tertiary_enrolment_3y_shift', 'src_tertiary_enrolment_4y_shift', 'src_tertiary_enrolment_5y_shift'
        ]
        dest_factors = dest_factors[new_col]
        
        #dic of predictors:
        i = 0
        predictors = {
                        0:[],
                        1:[],
                        2:[],
                        3:[],
                        4:[],
                        5:[]
        }
        for pred in dest_factors.columns.values[1:]:
            predictors[i].append(pred)
            if i != 5:
                i += 1
            else:
                i = 0
        # calculating corr and pvalue:
        c_dic = {}
        p_dic = {}
        t_dic = {}
        for k,lst in predictors.items():
            for pred in lst:
                #calculate:
                c_p_pair = pearsonr(dest_factors['dst_students_count'][k:],dest_factors[pred][k:])
                corr = c_p_pair[0]
                pvalue = c_p_pair[1]
                
                p_dic[pred] = pvalue
                c_dic[pred] = corr
                if pvalue is None or pvalue > 0.05:  #cannot reject H0, the corr is not sig
                    tested_corr = np.nan
                else:
                    tested_corr = corr
                t_dic[pred] = tested_corr
                
                #fill final_result_df:
                if k == 0:
                    pred_name = pred
                else:
                    sep = '_'+str(k)+'y'
                    pred_name = pred.split(sep)[0]
                    
                final_result_df = final_result_df.append(
                    pd.DataFrame({'Source':src,
                                  'Destination': dest,
                                  'Predictor':pred_name,
                                  'Shift': k,
                                  'Correlation': corr,
                                  'P-value': pvalue,
                                  'Tested Corralation': tested_corr
                                 }, index = [0]),
                    ignore_index=True)

        corr_dic[dest] = c_dic
        pvalue_dic[dest] = p_dic 
        tested_dic[dest] = t_dic
    
    corr_df = pd.DataFrame(corr_dic)
    corr_df = corr_df.reindex(index = new_col[1:])
    pvalue_df = pd.DataFrame(pvalue_dic)
    pvalue_df = pvalue_df.reindex(index = new_col[1:])
    tested_df = pd.DataFrame(tested_dic)
    tested_df = pvalue_df.reindex(index = new_col[1:])
    
    #save the corraltion table, p-value table:
    #corr_df.to_csv(path_or_buf = 'output/Source_Destination_Count_correlation_'+src+'.csv')
    #pvalue_df.to_csv(path_or_buf = 'output/Source_Destination_Count_pvalue_'+src+'.csv')


#save finel result table:
final_result_df.to_csv(path_or_buf = 'output/correlation_table_with_5_year_shift.csv')


/usr/local/lib/python3.7/site-packages/scipy/stats/stats.py:3399: PearsonRConstantInputWarning:

An input array is constant; the correlation coefficent is not defined.



  
    #plot:
    trace = go.Heatmap(z=corr_pvalues_df.values,
                        x=corr_pvalues_df.columns.values,
                        y=corr_pvalues_df.index.values,
                        colorscale='spectral',
                        reversescale=True)

    layout = go.Layout(
        title="Correlation Between Predcitors and Student Count from "+src,
        autosize=False,
        width=1000,
        height=2000,
        yaxis = dict(visible  = True,
                    constraintoward = 'left')
        )


    fig = go.Figure(data=trace, layout=layout)
    iplot(fig)
        